In [1]:
cd ..

/home/alberto/Work/course_interpretability_deep_learning


# Multi-omics stratification on PDAC patients

In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold
from sklearn.impute import SimpleImputer
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.tuner import Tuner
from pytorch_lightning.utilities.seed import isolate_rng
import optuna
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import torch
torch.set_float32_matmul_precision('high')

from src import settings
from utils import RemovePatientsWithZeros, RemovePatientsWithNaN, FeatureSelectionNMF, RemoveCorrelatedFeatures, RemoveFeaturesLowMAE
from optimization import Optimization

## Load dataset

In [3]:
methylation_data = pd.read_csv(settings.methylation_data_path, sep=";", index_col=0, decimal=",")
methylation_data.columns = methylation_data.columns.str.replace(".", "-")
methylation_data = methylation_data.T
methylation_data = methylation_data.astype(np.float32)
print("methylation_data.shape", methylation_data.shape)
methylation_data.head()

methylation_data.shape (153, 301195)


,cg00000029,cg00000236,cg00000289,cg00000292,cg00000321,cg00000622,cg00000658,cg00000714,cg00000721,cg00000734,...,ch.9.2262725R,ch.9.2285199R,ch.9.2298007R,ch.9.2473665R,ch.9.357218F,ch.9.377428R,ch.9.691424R,ch.9.837340R,ch.9.898515R,ch.9.991104F
TCGA-2J-AAB6,0.157951,0.836226,0.710511,0.560780,0.239194,0.016433,0.864604,0.087681,0.938775,0.061008,...,0.103136,0.053757,0.032478,NaN,0.064965,0.049776,0.115268,0.095954,0.084203,NaN
TCGA-2J-AAB8,0.300754,0.782242,0.574296,0.670286,0.424310,0.014747,0.885958,0.112524,0.930765,0.037198,...,0.028180,0.054483,0.022736,NaN,0.060835,0.036434,0.160082,0.059216,0.065342,0.166304
TCGA-2J-AAB9,0.257807,0.846522,0.534748,0.688073,0.295597,0.014649,0.895039,0.167297,0.940112,0.058407,...,0.059313,0.063187,0.032581,NaN,0.055342,0.069086,0.128546,0.120015,0.074940,NaN
TCGA-2J-AABA,0.239086,0.789457,0.474723,0.705372,0.530321,0.016919,0.884874,0.129581,0.910885,0.062167,...,0.122677,0.056068,0.023190,0.109351,0.056015,0.053238,0.082979,0.057172,0.045781,0.121676
TCGA-2J-AABE,0.168622,0.841684,0.591205,0.623799,0.322576,0.014408,0.898202,0.125415,0.941153,0.059365,...,0.046699,0.049177,0.032707,NaN,0.075854,0.062602,0.122072,0.082753,0.071240,NaN


In [4]:
rnaseq_data = pd.read_csv(settings.rnaseq_data_path, sep=";", index_col=0, decimal=",")
rnaseq_data = rnaseq_data.T
rnaseq_data = rnaseq_data.astype(np.float32)
print("rnaseq_data.shape", rnaseq_data.shape)
rnaseq_data.head()

rnaseq_data.shape (147, 20501)


,A1BG,A1CF,A2BP1,A2LD1,A2ML1,A2M,A4GALT,A4GNT,AAA1,AAAS,...,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,psiTPTE22,tAKR
TCGA-2J-AAB6,82.549698,8.187100,0.0000,163.122803,1815.789551,8517.444336,1121.052612,1.169600,1.1696,834.502930,...,14.619900,269.005798,1053.216431,0.5848,683.625671,11696.491211,869.005798,601.754395,26.315800,0.0000
TCGA-2J-AAB8,56.930698,33.842499,0.0000,185.814301,16.921301,14413.913086,392.949493,9.400700,0.9401,801.880127,...,35.722698,356.286713,829.142212,3.7603,680.611023,5829.377441,828.202087,609.165710,85.546402,0.0000
TCGA-2J-AAB9,105.787804,21.436199,1.0718,166.709503,642.015015,24311.779297,1125.401855,50.375099,0.0000,862.808105,...,57.877800,381.564789,936.763123,1.0718,646.302307,8094.319336,1083.601318,573.419128,30.010700,0.0000
TCGA-2J-AABA,99.345497,18.788200,0.0000,99.276703,873.649597,10302.006836,633.161072,6.262700,18.7882,623.767029,...,52.606899,293.721588,1511.820923,1.2525,945.670898,4829.810547,1364.646851,793.486816,31.313601,0.6263
TCGA-2J-AABE,79.401901,3.083100,0.0000,134.564499,74.610802,11076.861328,710.343811,35.147202,0.0000,702.327698,...,56.728802,431.632507,1069.215454,0.6166,564.205322,7464.775879,832.434082,468.629608,48.096199,0.0000


In [5]:
samples = methylation_data.index.intersection(rnaseq_data.index)
methylation_data = methylation_data.loc[samples]
rnaseq_data = rnaseq_data.loc[samples]
assert methylation_data.index.equals(rnaseq_data.index)

In [6]:
rnaseq_pipeline = make_pipeline(
    RemovePatientsWithZeros(threshold= 0.2, verbose= True),
    RemoveFeaturesLowMAE(percentage_to_keep= 0.5, verbose= True),
    RemoveCorrelatedFeatures(threshold = 0.85, verbose= True),
    FunctionTransformer(lambda x: np.log2(1 + x)),
    FeatureSelectionNMF(nmf = NMF(n_components= 50, max_iter=1000, random_state=settings.RANDOM_STATE), n_largest= 3, verbose= True),
    StandardScaler().set_output(transform= 'pandas'),
)
rnaseq_pipeline

Pipeline(steps=[('removepatientswithzeros',
                 RemovePatientsWithZeros(verbose=True)),
                ('removefeatureslowmae',
                 RemoveFeaturesLowMAE(percentage_to_keep=0.5, verbose=True)),
                ('removecorrelatedfeatures',
                 RemoveCorrelatedFeatures(threshold=0.85, verbose=True)),
                ('functiontransformer',
                 FunctionTransformer(func=<function <lambda> at 0x7fdd58be0d30>)),
                ('featureselectionnmf',
                 FeatureSelectionNMF(n_largest=3,
                                     nmf=NMF(max_iter=500, n_components=50,
                                             random_state=42),
                                     verbose=True)),
                ('standardscaler', StandardScaler())])

In [7]:
methylation_pipeline = make_pipeline(
    RemovePatientsWithNaN(threshold = 0.2, verbose= True),
    RemoveFeaturesLowMAE(percentage_to_keep= 0.1, verbose= True),
    # RemoveCorrelatedFeatures(threshold = 0.85, verbose= True),
    SimpleImputer(strategy= "mean").set_output(transform= 'pandas'),
    FeatureSelectionNMF(nmf = NMF(n_components= 25, max_iter=1000, random_state=settings.RANDOM_STATE), n_largest= 10, verbose= True),
    StandardScaler().set_output(transform= 'pandas'),
)
methylation_pipeline

Pipeline(steps=[('removepatientswithnan', RemovePatientsWithNaN(verbose=True)),
                ('removefeatureslowmae',
                 RemoveFeaturesLowMAE(percentage_to_keep=0.1, verbose=True)),
                ('simpleimputer', SimpleImputer()),
                ('featureselectionnmf',
                 FeatureSelectionNMF(n_largest=10,
                                     nmf=NMF(max_iter=1000, n_components=25,
                                             random_state=42),
                                     verbose=True)),
                ('standardscaler', StandardScaler())])

In [8]:
# import shutil
# shutil.rmtree("tensorboard/")
# shutil.rmtree("lightning_logs/")
# shutil.rmtree("checkpoints/")

In [9]:
studies = []
for train_index, test_index in KFold(n_splits=5, shuffle= True, random_state=settings.RANDOM_STATE).split(samples):
    rnaseq_train, rnaseq_test = rnaseq_data.iloc[train_index], rnaseq_data.iloc[test_index]
    methylation_train, methylation_test = methylation_data.iloc[train_index], methylation_data.iloc[test_index]
    study = optuna.create_study(direction="minimize")
    func_objective = lambda trial: Optimization.objective(trial= trial,
                                                          rnaseq_provtrain= rnaseq_train, methylation_provtrain= methylation_train,
                                                          rnaseq_test= rnaseq_test, methylation_test= methylation_test,
                                                          rnaseq_pipeline= rnaseq_pipeline, methylation_pipeline= methylation_pipeline)
    study.optimize(func_objective, n_trials=3, n_jobs= 3, show_progress_bar= True)
    studies.append(study)

[I 2023-05-23 17:47:45,313] A new study created in memory with name: no-name-162ad463-2a9a-4e41-bf7e-211a83cf4356
/home/alberto/anaconda3/envs/course_interpretability_dl/lib/python3.10/site-packages/optuna/progress_bar.py:56: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/3 [00:00<?, ?it/s]

RemovePatientsWithZeros keeping 17096 features
RemoveFeaturesLowMAE keeping 8548 features
RemoveCorrelatedFeatures keeping 6985 features
FeatureSelectionNMF keeping 800 features
RemovePatientsWithNaN keeping 300793 features
RemoveFeaturesLowMAE keeping 30079 features
FeatureSelectionNMF keeping 150 features


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/alberto/anaconda3/envs/course_interpretability_dl/lib/python3.10/site-packages/pytorch_lightning/loops/utilities.py:70: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
/home/alberto/anaconda3/envs/course_interpretability_dl/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:72: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_loss          │    0.6338875889778137     │
│        val_loss_0         │    0.6439158916473389     │
│        val_loss_1         │    0.6126509308815002     │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_loss          │    0.7267386317253113     │
│        val_loss_0         │     0.762497067451477     │
│        val_loss_1         │    0.6510149836540222     │
└───────────────────────────┴───────────────────────────┘

{'val_loss_0': 0.25405436754226685, 'val_loss_1': 0.3400305509567261, 'val_loss': 0.28163161873817444}
[0.025118864315095822, 0.003981071705534969, 0.012022644346174132, 0.025118864315095822, 0.012022644346174132]
[I 2023-05-23 18:51:14,394] Trial 2 finished with value: 0.6281437039375305 and parameters: {'num_features_rnaseq': 450, 'num_features_methylation': 425, 'num_layers': 1, 'num_units_0': 6}. Best is trial 2 with value: 0.6281437039375305.


In [19]:
study.trials_dataframe().iloc[:, 20:25]

,user_attrs_val_loss_1_list,user_attrs_val_loss_list,state
0,"[1.0443567037582397, 1.0365943908691406, 1.024...","[1.8352055549621582, 1.0732187032699585, 0.996...",COMPLETE
1,"[1.0120737552642822, 0.694180428981781, 0.9780...","[1.022976040840149, 0.739151656627655, 0.88281...",COMPLETE
2,"[0.6247801184654236, 0.6102632284164429, 0.552...","[0.6381224989891052, 0.6325669288635254, 0.581...",COMPLETE
